In [1]:
import os
import numpy as np
import cv2
import pandas as pd


from module.model import Model
from module.layers import *
from module.activations import ReLU, Softmax
from module.optimizers import Adam
from module.losses import CategoricalCrossentropy

from module.accuracy import Categorical_Accuracy
from module.utils import one_hot_encoding
from module.utils import train_test_split


In [2]:
data = pd.read_csv('data/digit-recognizer/train.csv') 
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
y = data['label'].to_numpy()
X = data.drop(['label'], axis=1).to_numpy()
X_train, y_train, X_test, y_test = train_test_split(X, y, test_size = 1000, random_state = 42, shuffle = True)

In [4]:
category = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
label_map = {cat: i for i, cat in enumerate(category)}

In [5]:
y_train = one_hot_encoding(y_train, category)
y_test = one_hot_encoding(y_test, category)

In [6]:
np.expand_dims(X_train, axis=-1).shape

(41000, 784, 1)

In [7]:
(782 - 2) // 2 + 1

391

In [8]:
# Instantiate the model
model = Model()


# Add layers
# model.add(Dense(X_train.shape[1], 128))
model.add(Conv1D(n_filters=128, filter_size=3, n_inputs=X_train.shape[1]))
model.add(ReLU())
model.add(MaxPool1D(pool_size=2, strides=2))
model.add(Flatten())
model.add(Dense((391)*128, 128))
model.add(ReLU())
model.add(Dense(128, 10))
model.add(Softmax())

# Set loss, optimizer and accuracy objects
model.set(
    loss=CategoricalCrossentropy(),
    optimizer=Adam(learning_rate=1e-3, decay=1e-3),
    accuracy=Categorical_Accuracy()
)

# Finalize the model
model.finalize()

# Train the model
model.train(np.expand_dims(X_train, axis=-1), y_train, validation_data=(X_test, y_test),
            epochs=10, batch_size=128, print_every=100)

TypeError: MaxPool1D.__init__() got an unexpected keyword argument 'stride'

In [ ]:
conv = Conv1D(n_filters=32, filter_size=3, n_inputs=X_train.shape[1])
conv.forward(np.expand_dims(X_train, axis=-1), training=True)
conv.output.shape

(41000, 782, 32)

In [ ]:
# Predict on the image
confidences = model.predict(X_test)

# Get prediction instead of confidence levels
predictions = model.output_layer_activation.predictions(confidences)

# Get label name from label index
prediction = label_map[predictions[2]]

print(f'predicted: {prediction}')
print(f'true_value: {np.argmax(y_test[2])}')

NameError: name 'model' is not defined

In [ ]:
np.argmax(confidences, axis=1)

array([8, 1, 9, 9, 8, 6, 2, 2, 7, 1, 6, 3, 1, 2, 7, 4, 3, 3, 6, 4, 9, 5,
       2, 6, 0, 0, 0, 8, 6, 3, 6, 6, 1, 5, 8, 6, 7, 3, 6, 1, 9, 7, 9, 6,
       8, 3, 4, 2, 7, 8, 4, 3, 3, 7, 1, 2, 6, 2, 9, 6, 4, 0, 4, 8, 5, 5,
       4, 3, 9, 3, 9, 4, 2, 8, 1, 6, 3, 9, 0, 3, 1, 7, 6, 7, 6, 1, 9, 5,
       3, 6, 9, 3, 7, 6, 6, 5, 2, 9, 3, 5, 4, 6, 0, 4, 1, 1, 1, 6, 8, 9,
       1, 1, 4, 7, 1, 0, 3, 6, 9, 8, 1, 3, 7, 3, 9, 6, 5, 7, 1, 0, 6, 5,
       6, 4, 2, 0, 9, 9, 9, 7, 2, 3, 6, 4, 0, 9, 9, 1, 7, 7, 1, 6, 8, 1,
       7, 6, 6, 5, 7, 9, 9, 0, 7, 9, 9, 1, 3, 6, 5, 8, 7, 1, 6, 7, 0, 9,
       3, 0, 8, 6, 0, 1, 9, 9, 2, 3, 7, 2, 6, 6, 4, 8, 9, 6, 0, 7, 4, 5,
       5, 6, 3, 1, 5, 3, 7, 9, 9, 0, 7, 4, 7, 5, 1, 1, 0, 2, 5, 3, 8, 1,
       3, 9, 3, 6, 7, 3, 1, 3, 4, 4, 0, 2, 7, 1, 4, 2, 1, 0, 5, 3, 9, 8,
       7, 6, 5, 1, 0, 5, 7, 9, 7, 1, 7, 7, 4, 2, 2, 3, 4, 5, 2, 6, 4, 1,
       8, 8, 5, 5, 5, 4, 8, 6, 6, 6, 1, 0, 6, 3, 1, 9, 9, 0, 8, 3, 9, 7,
       7, 2, 3, 1, 8, 2, 6, 1, 5, 1, 0, 0, 5, 3, 5,